In [21]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import torch
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
import torchvision.datasets as dset
import torchvision.transforms as T
from sophius.encode import Encoder
import seaborn as sns
from tqdm import tqdm
from sophius.train import train_on_gpu_ex
normalize = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),    
])
cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True, transform=normalize)
cifar_gpu = dload.cifar_to_gpu(cifar10)

encoder = Encoder()

Files already downloaded and verified


In [23]:
from sophius.estimate import LSTMRegressor
from sophius.utils import calc_model_flops

estimator = torch.load('../data/models/estimator_v2.pth').cpu()

def estimate_acc(model_tmpl):
    t = torch.tensor(encoder.model2vec(model_tmpl), dtype=torch.float32)
    return estimator(t).item()

In [24]:
model_gen = ConvModelGenerator(
    in_shape=(3, 32, 32), 
    out_shape=10, 
    conv_num=1, 
    lin_num=1,
)

model_tmpl = model_gen.generate_model_tmpl()
model = model_tmpl.instantiate_model().cuda()

print(model_tmpl)
flops = calc_model_flops(model, model_gen.in_shape)['flops']
print(round(flops / 1024 / 1024, 2), 'MFlops')

train_params = {
    'val_size': 10000,
    'batch_size': 256,
    'num_epoch': 5,
    'random_seed':42,
    'optimizer': 'AdamW',
    'opt_params': {
        'lr': 1e-3,
    },
    'scheduler': 'ExponentialLR',
    'sch_params': {
        'gamma': 0.95,
    },
}

# res = train_on_gpu_ex(
#     model=model,
#     dataset=cifar_gpu,
#     verbose=True,
#     **train_params,
# )
# print('val_acc* %.3f' % (estimate_acc(model_tmpl)))
# 
# sns.lineplot(res, x='epoch', y='train_acc', label='train')
# sns.lineplot(res, x='epoch', y='val_acc', label='val');

Conv2d       (16, 9, 9)     (4, 4)   (4, 4)  
LeakyReLU    (16, 9, 9)     (0.1)   
AvgPool2d    (16, 5, 5)     (3, 3)   (2, 2)  
GlobalAvgPool2d (16, 1, 1)    
Flatten      16            
Linear       10            

0.12 MFlops


In [25]:
from sophius.db import database, Experiments, Models, ModelEpochs

with database:
    database.create_tables([Experiments, Models, ModelEpochs])

exp_params = {**train_params, **{'in_shape': (3, 32, 32), 'out_shape': 10}}

exp, _ = Experiments.get_or_create(**exp_params)

In [26]:
model_gen = ConvModelGenerator(
    in_shape=(3, 32, 32), out_shape=10, 
    conv_num=3, lin_num=1
)

val_acc_threshold = 0.2

for i in tqdm(range(10)):
    model_tmpl = model_gen.generate_model_tmpl()
    model = model_tmpl.instantiate_model().cuda()
    
    if estimate_acc(model_tmpl) < val_acc_threshold:
        continue
        
    epoch_results = train_on_gpu_ex(
        model=model,
        dataset=cifar_gpu,
        verbose=False,
        **train_params,
    )
    
    model_info = calc_model_flops(model, model_gen.in_shape)
    model = Models.create(
        exp_id = exp.id,
        hash = encoder.model2hash(model_tmpl),
        flops = model_info['flops'],
        macs = model_info['macs'],
        params = model_info['params'],
        val_acc = round(epoch_results.val_acc.iloc[-10:].mean(), 5),
        train_acc = round(epoch_results.train_acc.iloc[-10:].mean(), 5),
        time = round(epoch_results.time.iloc[-1], 1),
    )
    
    
    for _, row in epoch_results.iterrows():
        ModelEpochs.create(
            exp_id = exp.id,
            model_id = model.id,
            epoch = row['epoch'],
            loss = row['loss'],
            train_acc = row['train_acc'],
            val_acc = row['val_acc'],
            time = row['time'],
        )
    

100%|██████████| 10/10 [01:38<00:00,  9.81s/it]


In [27]:
import pandas as pd

with database.connection() as conn:
    df = pd.read_sql('SELECT * from models', conn)
    

In [28]:
df[-10:]

,id,exp_id,hash,flops,macs,params,val_acc,train_acc,time
0,1,1,0080204920000000400000000010000001000000,4653888,2288064,30730,0.43397,0.43914,30.0
1,2,1,0084040510020000004260004000000001000000,61536,29088,1930,0.45196,0.45797,1.9
2,3,1,0082101208080000004850004000000000233000001000...,76896,24896,554,0.27370,0.27668,2.4
3,4,1,0088101110020000400000000082008908040000004890...,320192,154944,10811,0.43946,0.46470,4.3
4,5,1,0080880908080000004250004000000000293000001000...,318528,140352,2506,0.36474,0.37869,3.5
5,6,1,0088011208040000004460000010000001000000,313792,153680,699,0.27660,0.27642,3.0
6,7,1,00802809200000000024b00000802105080400000048a0...,2475456,1202304,926603,0.56244,0.58730,12.7
7,8,1,0084102210020000004520000025500000803012200000...,1916224,837504,5642,0.40695,0.41044,19.7
8,9,1,0081010908040000004460000088010a10080000004860...,1081872,531280,17779,0.51406,0.53291,4.4
9,10,1,00802052080800000044a000400000000010000001000000,14638848,7227264,30922,0.46663,0.47428,13.6


In [29]:
encoder.hash2model('00804821200000000044a000400000000080809210020000400000000024d0000080880a0804000001000000', (3, 32, 32), 10)

Conv2d       (128, 31, 31)  (2, 2)   (1, 1)  
ReLU         (128, 31, 31) 
MaxPool2d    (128, 16, 16)  (3, 3)   (2, 2)  
BatchNorm2d  (128, 16, 16) 
Conv2d       (96, 9, 9)     (6, 6)   (2, 2)  
LeakyReLU    (96, 9, 9)     (0.001) 
BatchNorm2d  (96, 9, 9)    
AvgPool2d    (96, 5, 5)     (3, 3)   (2, 2)  
Conv2d       (96, 2, 2)     (2, 2)   (3, 3)  
PReLU        (96, 2, 2)    
Flatten      384           
Linear       10            